In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, explained_variance_score, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load the dataset
data = pd.read_csv('processed_data.csv')

# Define feature columns and target column
feature_columns = [col for col in data.columns if col not in ['datetime', 'price_log', 'price_boxcox', 'timezone']]
target_column = 'price_log'

# Separate features and target
X = data[feature_columns]
y = data[target_column]

# Identify numerical and categorical columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object', 'bool']).columns.tolist()

# Preprocessing for numerical data: standard scaling
numerical_transformer = StandardScaler()

# Preprocessing for categorical data: one-hot encoding
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define the model
model = GradientBoostingRegressor(random_state=42)

# Create the pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)])

# Define the parameter grid for GridSearchCV
param_grid = {
    'model__n_estimators': [100, 200, 300],
    'model__learning_rate': [0.01, 0.05, 0.1],
    'model__max_depth': [3, 5, 7],
    'model__min_samples_split': [2, 5, 10],
    'model__min_samples_leaf': [1, 2, 4]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, 
                           cv=3, n_jobs=-1, scoring='neg_mean_squared_error', verbose=3)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit GridSearchCV
try:
    grid_search.fit(X_train, y_train)
    
    # Get the best parameters and the best model
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    # Make predictions with the best model
    y_pred = best_model.predict(X_test)

    # Evaluate the best model
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    adjusted_r2 = 1 - (1 - r2) * (len(y_test) - 1) / (len(y_test) - X_test.shape[1] - 1)
    explained_variance = explained_variance_score(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)

    # Print the best parameters and evaluation metrics
    print('Best Model Performance:')
    print(f'Best Parameters: {best_params}')
    print(f'Mean Squared Error (MSE): {mse}')
    print(f'R-squared (R²) Score: {r2}')
    print(f'Adjusted R-squared: {adjusted_r2}')
    print(f'Explained Variance Score: {explained_variance}')
    print(f'Mean Absolute Percentage Error (MAPE): {mape}')

except Exception as e:
    print(f"Error during GridSearchCV fitting: {e}")

# Additional debug information
print("GridSearchCV fit status:")
print(grid_search.cv_results_)


Fitting 3 folds for each of 243 candidates, totalling 729 fits
Best Model Performance:
Best Parameters: {'model__learning_rate': 0.1, 'model__max_depth': 3, 'model__min_samples_leaf': 1, 'model__min_samples_split': 10, 'model__n_estimators': 300}
Mean Squared Error (MSE): 0.00028630592247340923
R-squared (R²) Score: 0.9986954181455355
Adjusted R-squared: 0.9928972765701379
Explained Variance Score: 0.9987221805558546
Mean Absolute Percentage Error (MAPE): 0.0018312578781679181
GridSearchCV fit status:
{'mean_fit_time': array([0.20013309, 0.94580754, 1.82279555, 0.90624475, 1.47040391,
       2.15989288, 0.788891  , 1.45577359, 2.15956012, 0.77958361,
       1.48137323, 2.11468132, 0.79986183, 1.4727308 , 1.72737877,
       0.78224134, 0.67818435, 1.09008686, 0.27825832, 1.25597906,
       2.06481393, 0.76562134, 1.46408534, 2.32212671, 0.83609811,
       1.56215692, 2.29619519, 1.21574926, 2.50368142, 2.65989653,
       1.3146592 , 1.52433141, 2.68620928, 0.87648924, 2.0724167 ,
      